In [1]:
import pandas as pd
import urllib.request
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from haversine import haversine, Unit
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import pytz
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', None)

## 1) Extract FIgLib Data

In [2]:
# 1) Loop through each directory in http://hpwren.ucsd.edu/HPWREN-FIgLib/HPWREN-FIgLib-Data/
#    Each folder represents a fire event. Grab the entire name
base_url = "http://hpwren.ucsd.edu/HPWREN-FIgLib/HPWREN-FIgLib-Data/"
soup = BeautifulSoup(requests.get(base_url).text)

events = []

for a in soup.table.find_all('a'):
    events.append(a['href'])

#skips the first 4 links in the page (they are the table headers and the parent directory)
events = events[5:]

In [3]:
#    
# 2) Grab the full name of the +00000.jpg file
# This won't get all of the pics, as of 04/07/2022 9 events don't have the +00000.jpg suffix
#
pics = []
not_found = []
for idx,e in enumerate(events):
    e_url = f"{base_url}{e}"
    soup = BeautifulSoup(requests.get(e_url).text)
    found = False
    for a in soup.table.find_all('a'):
        a_href = a['href']
        
        if "00000.jpg" in a_href:
            pics.append((idx, a_href))
            found = True

    if found == False:
        not_found.append(e)

In [4]:
event_df = pd.DataFrame(events, columns=["event_name"])
event_df

,event_name
0,20160604_FIRE_rm-n-mobo-c/
1,20160604_FIRE_smer-tcs3-mobo-c/
2,20160619_FIRE_lp-e-iqeye/
3,20160619_FIRE_om-e-mobo-c/
4,20160619_FIRE_pi-s-mobo-c/
...,...
311,20210209_FIRE_tp-w-mobo-c/
312,20210302_FIRE_lp-e-mobo-c/
313,20210302_FIRE_lp-e-mobo-m/
314,20210319_FIRE_om-n-mobo-c/


In [5]:
pic_df = pd.DataFrame(pics, columns=["idx", "pic_name"])
pic_df = pic_df.set_index("idx")
pic_df

,pic_name
idx,
0,1465065600_+00000.jpg
1,1465065668_+00000.jpg
2,1466359453_+00000.jpg
3,1466359357_+00000.jpg
4,1466359567_+00000.jpg
...,...
311,1612894635_+00000.jpg
312,1614679618_+00000.jpg
313,1614679559_+00000.jpg


In [6]:
event_pic_df = event_df.merge(pic_df, left_index=True, right_index=True, how="inner")
event_pic_df["date"] = event_pic_df["event_name"].str[:8]
event_pic_df["camera_name"] = event_pic_df["event_name"].str.split("_").str[-1].str[:-1]

#Edge case: 2 camera names not following naming convention. Uses "-" instead of "_"
event_pic_df.loc[event_pic_df["camera_name"] == "FIRE-pi-s-mobo-c", ["camera_name"]] = "pi-s-mobo-c"
event_pic_df.loc[event_pic_df["camera_name"] == "FIRE-smer-tcs8-mobo-c", ["camera_name"]] = "smer-tcs8-mobo-c"

event_pic_df["time"] = event_pic_df["pic_name"].str.split("_").str[0]
event_pic_df["datetime"] = pd.to_datetime(event_pic_df["time"], unit="s", origin="unix", utc=True)
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00
...,...,...,...,...,...,...
311,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00
312,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00
313,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00
314,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00


In [7]:
event_pic_df["datetime"].min()

Timestamp('2016-06-04 18:40:00+0000', tz='UTC')

## 2) Camera Metadata Join

In [8]:
camera_metadata_df = pd.read_csv("camera_metadata - camera_metadata.csv")
camera_metadata_df

,camera_id,lat,lat_decimals,long,camera_name,hpweren_camera_description,lat_adjusted,long_adjusted,sample_image_url,camera_model,comment
0,hpwren0_unknown direction,32.550000,2,-117.120000,tje-1-mobo-c,NaN,NaN,NaN,NaN,NaN,NaN
1,hpwren1_north,33.159900,4,-116.808100,bm-n-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
2,hpwren1_east,33.159900,4,-116.808100,bm-e-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
3,hpwren1_south,33.159900,4,-116.808100,bm-s-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
4,hpwren1_west,33.159900,4,-116.808100,bm-w-mobo-c,Big Black Mountain,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1158,Axis-SYP,34.527107,6,-119.978905,syp-n-mobo-c,Santa Ynez Peak,NaN,NaN,NaN,NaN,NaN
1159,Axis-SYP,34.527107,6,-119.978905,syp-w-mobo-c,Santa Ynez Peak,NaN,NaN,NaN,NaN,NaN
1160,hpwren19_east,33.008700,4,-116.970900,wc-e-mobo-c,Mt. Woodson,NaN,NaN,NaN,NaN,NaN
1161,hpwren19_north,33.008700,4,-116.970900,wc-n-mobo-c,Mt. Woodson,NaN,NaN,NaN,NaN,NaN


In [9]:
camera_metadata_filtered_df = camera_metadata_df[["camera_name", "lat", "long", "lat_adjusted", "long_adjusted"]]
event_pic_df = event_pic_df.merge(camera_metadata_filtered_df, left_on="camera_name", right_on="camera_name", how="left")
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00,33.4008,-117.1905,NaN,NaN
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00,NaN,NaN,NaN,NaN
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00,32.7015,-116.7646,NaN,NaN
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00,32.5948,-116.8447,NaN,NaN
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00,32.7300,-116.5800,32.734673,-116.577751
...,...,...,...,...,...,...,...,...,...,...
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,NaN,NaN
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,NaN,NaN
304,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00,NaN,NaN,NaN,NaN
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,NaN,NaN


In [10]:
event_pic_df.loc[event_pic_df["lat_adjusted"].isna(), ["lat_adjusted"]] = event_pic_df["lat"]
event_pic_df.loc[event_pic_df["long_adjusted"].isna(), ["long_adjusted"]] = event_pic_df["long"]

In [11]:
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00,33.4008,-117.1905,33.400800,-117.190500
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00,NaN,NaN,NaN,NaN
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00,32.7015,-116.7646,32.701500,-116.764600
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00,32.5948,-116.8447,32.594800,-116.844700
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00,32.7300,-116.5800,32.734673,-116.577751
...,...,...,...,...,...,...,...,...,...,...
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,33.523500,-116.425700
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,32.701500,-116.764600
304,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00,NaN,NaN,NaN,NaN
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,32.594800,-116.844700


In [12]:
coords = [Point(xy) for xy in zip(event_pic_df['long_adjusted'], event_pic_df['lat_adjusted'])]
event_pic_df = GeoDataFrame(event_pic_df, crs = "EPSG:4326", geometry = coords) 
event_pic_df = event_pic_df.to_crs('EPSG:3310')
event_pic_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted,geometry
0,20160604_FIRE_rm-n-mobo-c/,1465065600_+00000.jpg,20160604,rm-n-mobo-c,1465065600,2016-06-04 18:40:00+00:00,33.4008,-117.1905,33.400800,-117.190500,POINT (261485.673 -508733.904)
1,20160604_FIRE_smer-tcs3-mobo-c/,1465065668_+00000.jpg,20160604,smer-tcs3-mobo-c,1465065668,2016-06-04 18:41:08+00:00,NaN,NaN,NaN,NaN,POINT (nan nan)
2,20160619_FIRE_lp-e-iqeye/,1466359453_+00000.jpg,20160619,lp-e-iqeye,1466359453,2016-06-19 18:04:13+00:00,32.7015,-116.7646,32.701500,-116.764600,POINT (303754.080 -584902.039)
3,20160619_FIRE_om-e-mobo-c/,1466359357_+00000.jpg,20160619,om-e-mobo-c,1466359357,2016-06-19 18:02:37+00:00,32.5948,-116.8447,32.594800,-116.844700,POINT (296629.874 -596963.817)
4,20160619_FIRE_pi-s-mobo-c/,1466359567_+00000.jpg,20160619,pi-s-mobo-c,1466359567,2016-06-19 18:06:07+00:00,32.7300,-116.5800,32.734673,-116.577751,POINT (321156.327 -580614.901)
...,...,...,...,...,...,...,...,...,...,...,...
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,33.523500,-116.425700,POINT (332124.332 -492745.685)
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,32.701500,-116.764600,POINT (303754.080 -584902.039)
304,20210302_FIRE_lp-e-mobo-m/,1614679559_+00000.jpg,20210302,lp-e-mobo-m,1614679559,2021-03-02 10:05:59+00:00,NaN,NaN,NaN,NaN,POINT (nan nan)
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,32.594800,-116.844700,POINT (296629.874 -596963.817)


In [13]:
event_pic_df.to_csv("event_pic.csv", index=False)

## 3) WFABBA Join

In [16]:
# time_filter = event_pic_df["datetime"] >= "2021-01-20"
time_filter = event_pic_df["datetime"] >= "2020-01-01"
lat_filter = ~event_pic_df["lat"].isna()
overlap_events_df = event_pic_df[time_filter & lat_filter]
overlap_events_df

,event_name,pic_name,date,camera_name,time,datetime,lat,long,lat_adjusted,long_adjusted,geometry
208,20200202_FIRE_hp-w-mobo-c/,1580667991_+00000.jpg,20200202,hp-w-mobo-c,1580667991,2020-02-02 18:26:31+00:00,33.3600,-116.8400,33.363018,-116.83622,POINT (294587.374 -511880.655)
209,20200205_FIRE_hp-w-mobo-c/,1580927071_+00000.jpg,20200205,hp-w-mobo-c,1580927071,2020-02-05 18:24:31+00:00,33.3600,-116.8400,33.363018,-116.83622,POINT (294587.374 -511880.655)
210,20200206_FIRE_ml-s-mobo-c/,1581011115_+00000.jpg,20200206,ml-s-mobo-c,1581011115,2020-02-06 17:45:15+00:00,32.8922,-116.4209,32.892200,-116.42090,POINT (335210.472 -562636.766)
211,20200226_FIRE_rm-e-mobo-c/,1582742582_+00000.jpg,20200226,rm-e-mobo-c,1582742582,2020-02-26 18:43:02+00:00,33.4008,-117.1905,33.400800,-117.19050,POINT (261485.673 -508733.904)
212,20200304_FIRE_rm-w-mobo-c/,1583345046_+00000.jpg,20200304,rm-w-mobo-c,1583345046,2020-03-04 18:04:06+00:00,33.4008,-117.1905,33.400800,-117.19050,POINT (261485.673 -508733.904)
...,...,...,...,...,...,...,...,...,...,...,...
301,20210209_FIRE_hp-e-mobo-c/,1612892967_+00000.jpg,20210209,hp-e-mobo-c,1612892967,2021-02-09 17:49:27+00:00,33.3600,-116.8400,33.363018,-116.83622,POINT (294587.374 -511880.655)
302,20210209_FIRE_tp-w-mobo-c/,1612894635_+00000.jpg,20210209,tp-w-mobo-c,1612894635,2021-02-09 18:17:15+00:00,33.5235,-116.4257,33.523500,-116.42570,POINT (332124.332 -492745.685)
303,20210302_FIRE_lp-e-mobo-c/,1614679618_+00000.jpg,20210302,lp-e-mobo-c,1614679618,2021-03-02 10:06:58+00:00,32.7015,-116.7646,32.701500,-116.76460,POINT (303754.080 -584902.039)
305,20210319_FIRE_om-n-mobo-c/,1616211942_+00000.jpg,20210319,om-n-mobo-c,1616211942,2021-03-20 03:45:42+00:00,32.5948,-116.8447,32.594800,-116.84470,POINT (296629.874 -596963.817)


In [35]:
#read in GOES 16 inputs
wfabba_goes_16_2019_df = pd.read_csv("goes-16_2019.csv")
wfabba_goes_16_part1_df = pd.read_csv("goes-16_2020_Part1.csv")
wfabba_goes_16_part2_df = pd.read_csv("goes-16_2020_Part2.csv")
wfabba_goes_16_part3_df = pd.read_csv("goes-16_2020_Part3.csv")
wfabba_goes_16_part4_df = pd.read_csv("goes-16_2020_Part4.csv")
wfabba_goes_16_jan_2021_df = pd.read_csv("goes-16_2021_Jan.csv")
wfabba_goes_16_2021_df = pd.read_csv("goes-16.csv") #2021 detections

In [36]:
wfabba_goes_16_2019_df = wfabba_goes_16_2019_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_part1_df = wfabba_goes_16_part1_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_part2_df = wfabba_goes_16_part2_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_part3_df = wfabba_goes_16_part3_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_part4_df = wfabba_goes_16_part4_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_jan_2021_df = wfabba_goes_16_jan_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element", "FD_X", "FD_Y"]) #2021 detections

In [37]:
print(len(wfabba_goes_16_2021_df))
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df[wfabba_goes_16_2021_df["Timestamp"] >= "2021-02-01"]
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df.reset_index()
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df.drop(columns=["index"])
print(len(wfabba_goes_16_2021_df))
wfabba_goes_16_2021_df

179504
177751


,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco
0,6_6_001g,2021-07-18 22:01:19,GOES-16,FM?,GOES-16,GOES-16,f2021199220119.rev.6_6_001g.FDCC.GOES-16.txt,-9999,31.8610,-113.3841,10,75.4,149722.0,435.4,8048279,327.115,301.904,320.516,299.474,33.296,55.546,-9999,51
1,6_6_001g,2021-07-18 22:01:19,GOES-16,FM?,GOES-16,GOES-16,f2021199220119.rev.6_6_001g.FDCC.GOES-16.txt,-9999,31.7891,-113.3725,10,58.6,174114.0,416.2,8037276,325.240,300.849,320.484,299.520,33.294,55.493,-9999,51
2,6_6_001g,2021-07-18 22:01:19,GOES-16,FM?,GOES-16,GOES-16,f2021199220119.rev.6_6_001g.FDCC.GOES-16.txt,-9999,31.7873,-113.3403,10,76.2,209385.0,419.8,8030192,326.458,300.878,320.484,299.520,33.321,55.467,-9999,51
3,6_6_001g,2021-07-18 17:41:19,GOES-16,FM?,GOES-16,GOES-16,f2021199174119.rev.6_6_001g.FDCC.GOES-16.txt,-9999,32.7304,-116.6958,15,88.7,-9999.0,-9999.0,8858966,321.305,301.791,313.196,297.777,31.178,58.662,-9999,46
4,6_6_001g,2021-07-18 03:46:19,GOES-16,FM?,GOES-16,GOES-16,f2021199034619.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.9892,-114.6384,15,-9999.0,-9999.0,-9999.0,8622060,302.518,292.463,291.827,289.549,100.889,57.824,-9999,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177746,6_6_001g,2021-02-18 21:36:02,GOES-16,FM?,GOES-16,GOES-16,f2021049213602.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.1127,-115.4588,10,146.8,34206.0,580.3,8648794,318.263,298.127,299.334,296.677,50.981,57.918,-9999,37
177747,6_6_001g,2021-02-18 11:10:06,GOES-16,FM?,GOES-16,GOES-16,f2021049111006.rev.6_6_001g.FDCF.GOES-16.txt,-9999,34.0118,-117.6624,13,59.8,-9999.0,-9999.0,9306856,293.623,280.491,277.945,278.480,132.527,60.152,-9999,40
177748,6_6_001g,2021-02-18 11:16:02,GOES-16,FM?,GOES-16,GOES-16,f2021049111602.rev.6_6_001g.FDCC.GOES-16.txt,-9999,34.0118,-117.6624,13,57.9,-9999.0,-9999.0,9305936,293.251,280.559,277.939,278.491,131.330,60.152,-9999,40
177749,6_6_001g,2021-02-18 17:10:05,GOES-16,FM?,GOES-16,GOES-16,f2021049171005.rev.6_6_001g.FDCF.GOES-16.txt,-9999,35.1511,-116.0966,15,-9999.0,-9999.0,-9999.0,9140962,309.622,286.542,297.936,287.448,61.695,59.632,-9999,8


In [38]:
# GOES 16 operations
wfabba_goes_16_df = pd.concat([wfabba_goes_16_2019_df,wfabba_goes_16_part1_df,wfabba_goes_16_part2_df,wfabba_goes_16_part3_df,wfabba_goes_16_part4_df, wfabba_goes_16_jan_2021_df, wfabba_goes_16_2021_df])
# wfabba_goes_16_df = pd.concat([wfabba_goes_16_2019_df])
# wfabba_goes_16_df = pd.concat([wfabba_goes_16_part1_df,wfabba_goes_16_part2_df,wfabba_goes_16_part3_df,wfabba_goes_16_part4_df])
wfabba_goes_16_df["timestamp_converted"] = pd.to_datetime(wfabba_goes_16_df["Timestamp"], infer_datetime_format=True, origin="unix", utc=True)
wfabba_goes_16_df = wfabba_goes_16_df.reset_index()
wfabba_goes_16_df = wfabba_goes_16_df.drop(columns=["index"])
wfabba_goes_16_df

,index,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,timestamp_converted
0,0,6_5_012g,2019-06-01 23:36:41,GOES-16,FM1,C,FDCC,f2019152233641.rev.6_5_012g.FDCC.GOES-16,-9999,34.1101,-117.2837,15,46.0,-9999.0,-9999.0,11478677,307.035,286.865,302.505,292.199,51.492,59.916,-9999,1,2019-06-01 23:36:41+00:00
1,1,6_5_012g,2019-06-01 23:36:41,GOES-16,FM1,C,FDCC,f2019152233641.rev.6_5_012g.FDCC.GOES-16,-9999,32.9690,-113.4948,10,52.6,50251.0,476.1,9680612,317.625,308.088,313.579,308.984,54.716,56.320,-9999,47,2019-06-01 23:36:41+00:00
2,2,6_5_012g,2019-06-01 20:01:41,GOES-16,FM1,C,FDCC,f2019152200141.rev.6_5_012g.FDCC.GOES-16,-9999,33.3033,-113.0092,15,12.9,-9999.0,-9999.0,9610866,324.175,308.601,323.940,314.648,13.367,56.164,-9999,51,2019-06-01 20:01:41+00:00
3,3,6_5_012g,2019-06-01 00:31:41,GOES-16,FM1,C,FDCC,f2019152003141.rev.6_5_012g.FDCC.GOES-16,-9999,34.8145,-113.5976,15,55.6,-9999.0,-9999.0,10216718,308.467,297.128,302.069,299.778,65.685,57.570,-9999,51,2019-06-01 00:31:41+00:00
4,4,6_5_012g,2019-06-01 00:31:41,GOES-16,FM1,C,FDCC,f2019152003141.rev.6_5_012g.FDCC.GOES-16,-9999,34.8947,-118.9655,15,71.9,-9999.0,-9999.0,12601857,301.172,281.546,292.217,289.893,61.326,61.659,-9999,91,2019-06-01 00:31:41+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384444,177746,6_6_001g,2021-02-18 21:36:02,GOES-16,FM?,GOES-16,GOES-16,f2021049213602.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.1127,-115.4588,10,146.8,34206.0,580.3,8648794,318.263,298.127,299.334,296.677,50.981,57.918,-9999,37,2021-02-18 21:36:02+00:00
384445,177747,6_6_001g,2021-02-18 11:10:06,GOES-16,FM?,GOES-16,GOES-16,f2021049111006.rev.6_6_001g.FDCF.GOES-16.txt,-9999,34.0118,-117.6624,13,59.8,-9999.0,-9999.0,9306856,293.623,280.491,277.945,278.480,132.527,60.152,-9999,40,2021-02-18 11:10:06+00:00
384446,177748,6_6_001g,2021-02-18 11:16:02,GOES-16,FM?,GOES-16,GOES-16,f2021049111602.rev.6_6_001g.FDCC.GOES-16.txt,-9999,34.0118,-117.6624,13,57.9,-9999.0,-9999.0,9305936,293.251,280.559,277.939,278.491,131.330,60.152,-9999,40,2021-02-18 11:16:02+00:00
384447,177749,6_6_001g,2021-02-18 17:10:05,GOES-16,FM?,GOES-16,GOES-16,f2021049171005.rev.6_6_001g.FDCF.GOES-16.txt,-9999,35.1511,-116.0966,15,-9999.0,-9999.0,-9999.0,9140962,309.622,286.542,297.936,287.448,61.695,59.632,-9999,8,2021-02-18 17:10:05+00:00


In [39]:
#read in GOES 17 inputs
wfabba_goes_17_2019_df = pd.read_csv("goes-17_2019.csv")
wfabba_goes_17_part1_df = pd.read_csv("goes-17_2020_Part1.csv")
wfabba_goes_17_part2_df = pd.read_csv("goes-17_2020_Part2.csv")
wfabba_goes_17_part3_df = pd.read_csv("goes-17_2020_Part3.csv")
wfabba_goes_17_part4_df = pd.read_csv("goes-17_2020_Part4.csv")
wfabba_goes_17_part5_df = pd.read_csv("goes-17_2020_Part5.csv")
wfabba_goes_17_part6_df = pd.read_csv("goes-17_2020_Part6.csv")
wfabba_goes_17_jan_2021_df = pd.read_csv("goes-17_2021_Jan.csv")
wfabba_goes_17_1_df = pd.read_csv("goes-17_1.csv")
wfabba_goes_17_2_df = pd.read_csv("goes-17_2.csv")

In [40]:
wfabba_goes_17_2019_df = wfabba_goes_17_2019_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_part1_df = wfabba_goes_17_part1_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_part2_df = wfabba_goes_17_part2_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_part3_df = wfabba_goes_17_part3_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_part4_df = wfabba_goes_17_part4_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_part5_df = wfabba_goes_17_part5_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_part6_df = wfabba_goes_17_part6_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_jan_2021_df = wfabba_goes_17_jan_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_1_df = wfabba_goes_17_1_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element", "FD_X", "FD_Y"]) #2021 detections
wfabba_goes_17_2_df = wfabba_goes_17_2_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element", "FD_X", "FD_Y"]) #2021 detections

In [41]:
print(len(wfabba_goes_17_1_df))
wfabba_goes_17_1_df = wfabba_goes_17_1_df[wfabba_goes_17_1_df["Timestamp"] >= "2021-02-01"]
wfabba_goes_17_1_df = wfabba_goes_17_1_df.reset_index()
wfabba_goes_17_1_df = wfabba_goes_17_1_df.drop(columns=["index"])
print(len(wfabba_goes_17_1_df))
wfabba_goes_17_1_df

1004954
957233


,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco
0,6_6_001g,2021-02-01 22:40:25,GOES-17,FM?,GOES-17,GOES-17,f2021032224025.rev.6_6_001g.FDCM1.GOES-17.txt,-9999,35.2964,-120.0334,15.0,14.8,-9999.0,-9999.0,6183326,298.099,276.991,294.628,287.841,62.988,45.090,-9999,43
1,6_6_001g,2021-02-01 22:36:17,GOES-17,FM?,GOES-17,GOES-17,f2021032223617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,35.9931,-119.3861,15.0,7.7,-9999.0,-9999.0,6310942,299.865,281.456,298.637,291.350,63.342,46.063,-9999,37
2,6_6_001g,2021-02-01 22:36:17,GOES-17,FM?,GOES-17,GOES-17,f2021032223617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,35.2964,-120.0334,15.0,13.0,-9999.0,-9999.0,6184617,297.686,274.012,294.697,287.940,62.470,45.090,-9999,43
3,6_6_001g,2021-02-01 17:58:25,GOES-17,FM?,GOES-17,GOES-17,f2021032175825.rev.6_6_001g.FDCM1.GOES-17.txt,-9999,36.1203,-120.1433,15.0,13.4,-9999.0,-9999.0,6284671,296.458,277.696,293.249,287.638,62.185,45.865,-9999,40
4,6_6_001g,2021-02-01 22:38:25,GOES-17,FM?,GOES-17,GOES-17,f2021032223825.rev.6_6_001g.FDCM1.GOES-17.txt,-9999,35.2964,-120.0334,15.0,12.2,-9999.0,-9999.0,6183326,298.235,275.889,295.553,286.723,62.728,45.090,-9999,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957228,6_6_001g,2021-08-31 19:00:59,GOES-17,FM?,GOES-17,GOES-17,f2021243190059.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,32.7306,-116.6794,15.0,75.3,-9999.0,-9999.0,6078084,321.315,306.209,309.984,299.454,25.977,44.253,-9999,46
957229,6_6_001g,2021-08-31 20:03:55,GOES-17,FM?,GOES-17,GOES-17,f2021243200355.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,33.2583,-116.6280,15.0,100.0,-9999.0,-9999.0,6142462,324.114,309.973,310.721,301.029,24.500,44.772,-9999,46
957230,6_6_001g,2021-08-31 06:15:35,GOES-17,FM?,GOES-17,GOES-17,f2021243061535.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.6689,-112.7729,15.0,-9999.0,-9999.0,-9999.0,6244888,288.332,279.576,276.109,276.140,135.367,45.593,-9999,51
957231,6_6_001g,2021-08-31 06:15:35,GOES-17,FM?,GOES-17,GOES-17,f2021243061535.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.5504,-112.8116,15.0,-9999.0,-9999.0,-9999.0,6228500,289.036,280.537,277.266,277.273,135.457,45.468,-9999,51


In [42]:
print(len(wfabba_goes_17_2_df))
wfabba_goes_17_2_df = wfabba_goes_17_2_df[wfabba_goes_17_2_df["Timestamp"] >= "2021-02-01"]
wfabba_goes_17_2_df = wfabba_goes_17_2_df.reset_index()
wfabba_goes_17_2_df = wfabba_goes_17_2_df.drop(columns=["index"])
print(len(wfabba_goes_17_2_df))
wfabba_goes_17_2_df

509994
509994


,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco
0,6_6_001g,2021-09-01 21:55:55,GOES-17,FM?,GOES-17,GOES-17,f2021244215555.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,33.8971,-114.3734,12.0,64.1,-9999.0,-9999.0,6381689,320.705,299.385,313.072,298.191,40.515,46.592,-9999,51
1,6_6_001g,2021-09-01 15:01:55,GOES-17,FM?,GOES-17,GOES-17,f2021244150155.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,34.7891,-112.3438,15.0,22.6,-9999.0,-9999.0,6664578,296.889,282.224,291.465,287.582,66.147,48.541,-9999,51
2,6_6_001g,2021-09-01 02:59:49,GOES-17,FM?,GOES-17,GOES-17,f2021244025949.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,34.5995,-114.5459,15.0,-9999.0,-9999.0,-9999.0,6456915,288.654,277.415,276.942,276.753,101.378,47.130,-9999,51
3,6_6_001g,2021-09-01 02:59:49,GOES-17,FM?,GOES-17,GOES-17,f2021244025949.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,34.6005,-114.5201,15.0,-9999.0,-9999.0,-9999.0,6458958,288.397,279.886,276.670,276.479,101.398,47.145,-9999,51
4,6_6_001g,2021-09-01 02:59:49,GOES-17,FM?,GOES-17,GOES-17,f2021244025949.rev.6_6_001g.FDCM2.GOES-17.txt,-9999,34.5746,-114.5542,15.0,-9999.0,-9999.0,-9999.0,6452124,291.910,283.636,276.998,276.799,101.380,47.103,-9999,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509989,6_6_001g,2022-02-10 00:40:31,GOES-17,FM?,GOES-17,GOES-17,f2022041004031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.2105,-115.7500,12.0,34.9,-9999.0,-9999.0,5971248,301.612,290.239,291.898,291.449,81.850,43.385,-9999,22
509990,6_6_001g,2022-02-10 04:36:17,GOES-17,FM?,GOES-17,GOES-17,f2022041043617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,31.2105,-115.7500,10.0,38.4,50293.0,457.2,5971737,296.361,284.100,281.799,282.391,131.324,43.385,-9999,22
509991,6_6_001g,2022-02-10 00:20:31,GOES-17,FM?,GOES-17,GOES-17,f2022041002031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.2105,-115.7500,12.0,55.8,-9999.0,-9999.0,5971248,308.342,292.378,294.698,293.493,77.944,43.385,-9999,22
509992,6_6_001g,2022-02-10 00:16:17,GOES-17,FM?,GOES-17,GOES-17,f2022041001617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,31.2105,-115.7500,12.0,40.6,-9999.0,-9999.0,5971737,305.574,292.654,295.230,293.878,77.174,43.385,-9999,22


In [43]:
# GOES 17 operations
wfabba_goes_17_df = pd.concat([wfabba_goes_17_2019_df,wfabba_goes_17_part1_df, wfabba_goes_17_part2_df,wfabba_goes_17_part3_df,wfabba_goes_17_part4_df,wfabba_goes_17_part5_df,wfabba_goes_17_part6_df,wfabba_goes_17_jan_2021_df,wfabba_goes_17_1_df,wfabba_goes_17_2_df])
# wfabba_goes_17_df = pd.concat([wfabba_goes_17_2019_df])
# wfabba_goes_17_df = pd.concat([wfabba_goes_17_part1_df, wfabba_goes_17_part2_df,wfabba_goes_17_part3_df,wfabba_goes_17_part4_df,wfabba_goes_17_part5_df,wfabba_goes_17_part6_df])

wfabba_goes_17_df["timestamp_converted"] = pd.to_datetime(wfabba_goes_17_df["Timestamp"], infer_datetime_format=True, origin="unix", utc=True)
wfabba_goes_17_df = wfabba_goes_17_df.reset_index()
wfabba_goes_17_df = wfabba_goes_17_df.drop(columns=["index"])
wfabba_goes_17_df

,index,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,timestamp_converted
0,0,6_5_012g,2019-06-01 18:58:57,GOES-16,FM2,M2,FDCM2,f2019152185857.rev.6_5_012g.FDCM2.GOES-17,-9999,32.5523,-114.9965,10.0,50.6,17180.0,548.7,6448748,323.734,311.935,317.761,311.218,13.606,45.030,-9999,24,2019-06-01 18:58:57+00:00
1,1,6_5_012g,2019-06-01 18:52:57,GOES-16,FM2,M2,FDCM2,f2019152185257.rev.6_5_012g.FDCM2.GOES-17,-9999,32.7453,-115.5775,15.0,61.8,-9999.0,-9999.0,6415740,324.352,307.847,316.439,308.990,14.963,44.874,-9999,37,2019-06-01 18:52:57+00:00
2,2,6_5_012g,2019-06-01 22:06:57,GOES-16,FM2,M2,FDCM2,f2019152220657.rev.6_5_012g.FDCM2.GOES-17,-9999,32.9714,-116.6118,15.0,9.8,-9999.0,-9999.0,6345534,303.715,288.028,302.233,295.853,33.402,44.514,-9999,22,2019-06-01 22:06:57+00:00
3,3,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7783,-118.4323,12.0,440.7,-9999.0,-9999.0,6427847,351.035,307.493,316.979,307.734,25.775,45.308,-9999,51,2019-06-01 18:08:57+00:00
4,4,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7791,-118.4075,10.0,464.5,2077.0,1476.8,6429915,353.142,308.442,316.979,307.734,25.757,45.321,-9999,51,2019-06-01 18:08:57+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769962,509989,6_6_001g,2022-02-10 00:40:31,GOES-17,FM?,GOES-17,GOES-17,f2022041004031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.2105,-115.7500,12.0,34.9,-9999.0,-9999.0,5971248,301.612,290.239,291.898,291.449,81.850,43.385,-9999,22,2022-02-10 00:40:31+00:00
2769963,509990,6_6_001g,2022-02-10 04:36:17,GOES-17,FM?,GOES-17,GOES-17,f2022041043617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,31.2105,-115.7500,10.0,38.4,50293.0,457.2,5971737,296.361,284.100,281.799,282.391,131.324,43.385,-9999,22,2022-02-10 04:36:17+00:00
2769964,509991,6_6_001g,2022-02-10 00:20:31,GOES-17,FM?,GOES-17,GOES-17,f2022041002031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.2105,-115.7500,12.0,55.8,-9999.0,-9999.0,5971248,308.342,292.378,294.698,293.493,77.944,43.385,-9999,22,2022-02-10 00:20:31+00:00
2769965,509992,6_6_001g,2022-02-10 00:16:17,GOES-17,FM?,GOES-17,GOES-17,f2022041001617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,31.2105,-115.7500,12.0,40.6,-9999.0,-9999.0,5971737,305.574,292.654,295.230,293.878,77.174,43.385,-9999,22,2022-02-10 00:16:17+00:00


In [44]:
print(wfabba_goes_16_df.shape)
print(wfabba_goes_17_df.shape)

wfabba_goes_16_df = wfabba_goes_16_df[(wfabba_goes_16_df["Code"] != 15) & (wfabba_goes_16_df["Code"] != 35)]
wfabba_goes_17_df = wfabba_goes_17_df[(wfabba_goes_17_df["Code"] != 15) & (wfabba_goes_17_df["Code"] != 35)]

print(wfabba_goes_16_df.shape)
print(wfabba_goes_17_df.shape)

(384449, 25)
(2769967, 25)
(251783, 25)
(1553776, 25)


In [ ]:
#Convert the coordinates of WFABBA GOES-17 and GOES-16 from EPSG 4326 to EPSG 3310 to allow for distance calculations down to the meter

In [45]:
#Broke down the coordinate conversion steps for GOES 17 for easier diagnosis and prevent kernel killing
coords = [Point(xy) for xy in zip(wfabba_goes_17_df['Longitude'], wfabba_goes_17_df['Latitude'])]
print("Checkpoint A")

Checkpoint A


In [46]:
wfabba_goes_17_df = GeoDataFrame(wfabba_goes_17_df, crs = "EPSG:4326", geometry = coords) 
print("Checkpoint B")
wfabba_goes_17_df

Checkpoint B


,index,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,timestamp_converted,geometry
0,0,6_5_012g,2019-06-01 18:58:57,GOES-16,FM2,M2,FDCM2,f2019152185857.rev.6_5_012g.FDCM2.GOES-17,-9999,32.5523,-114.9965,10.0,50.6,17180.0,548.7,6448748,323.734,311.935,317.761,311.218,13.606,45.030,-9999,24,2019-06-01 18:58:57+00:00,POINT (-114.99650 32.55230)
3,3,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7783,-118.4323,12.0,440.7,-9999.0,-9999.0,6427847,351.035,307.493,316.979,307.734,25.775,45.308,-9999,51,2019-06-01 18:08:57+00:00,POINT (-118.43230 34.77830)
4,4,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7791,-118.4075,10.0,464.5,2077.0,1476.8,6429915,353.142,308.442,316.979,307.734,25.757,45.321,-9999,51,2019-06-01 18:08:57+00:00,POINT (-118.40750 34.77910)
5,5,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7335,-118.2973,12.0,107.0,-9999.0,-9999.0,6433137,328.538,308.442,317.586,308.300,25.656,45.329,-9999,46,2019-06-01 18:08:57+00:00,POINT (-118.29730 34.73350)
6,6,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7086,-118.3041,10.0,108.1,16963.0,619.1,6428921,329.336,309.062,317.245,308.025,25.651,45.302,-9999,46,2019-06-01 18:08:57+00:00,POINT (-118.30410 34.70860)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2769962,509989,6_6_001g,2022-02-10 00:40:31,GOES-17,FM?,GOES-17,GOES-17,f2022041004031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.2105,-115.7500,12.0,34.9,-9999.0,-9999.0,5971248,301.612,290.239,291.898,291.449,81.850,43.385,-9999,22,2022-02-10 00:40:31+00:00,POINT (-115.75000 31.21050)
2769963,509990,6_6_001g,2022-02-10 04:36:17,GOES-17,FM?,GOES-17,GOES-17,f2022041043617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,31.2105,-115.7500,10.0,38.4,50293.0,457.2,5971737,296.361,284.100,281.799,282.391,131.324,43.385,-9999,22,2022-02-10 04:36:17+00:00,POINT (-115.75000 31.21050)
2769964,509991,6_6_001g,2022-02-10 00:20:31,GOES-17,FM?,GOES-17,GOES-17,f2022041002031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,31.2105,-115.7500,12.0,55.8,-9999.0,-9999.0,5971248,308.342,292.378,294.698,293.493,77.944,43.385,-9999,22,2022-02-10 00:20:31+00:00,POINT (-115.75000 31.21050)
2769965,509992,6_6_001g,2022-02-10 00:16:17,GOES-17,FM?,GOES-17,GOES-17,f2022041001617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,31.2105,-115.7500,12.0,40.6,-9999.0,-9999.0,5971737,305.574,292.654,295.230,293.878,77.174,43.385,-9999,22,2022-02-10 00:16:17+00:00,POINT (-115.75000 31.21050)


In [47]:
#took geometry slice of wfabba_goes_17_df since there were too many columns, which killed the kernel
wfabba_goes_17_df_geom = wfabba_goes_17_df["geometry"]
wfabba_goes_17_df_geom = wfabba_goes_17_df_geom.to_crs('EPSG:3310')
wfabba_goes_17_df_geom

0          POINT (470495.929 -594187.763)
3          POINT (143396.998 -358615.563)
4          POINT (145663.743 -358488.961)
5          POINT (155833.437 -363374.892)
6          POINT (155260.597 -366149.588)
                        ...              
2769962    POINT (406343.478 -745894.014)
2769963    POINT (406343.478 -745894.014)
2769964    POINT (406343.478 -745894.014)
2769965    POINT (406343.478 -745894.014)
2769966    POINT (406343.478 -745894.014)
Name: geometry, Length: 1553776, dtype: geometry

In [48]:
#put the converted geometry slice back into wfabba_goes_17_df
wfabba_goes_17_df["geometry"] = wfabba_goes_17_df_geom
wfabba_goes_17_df[["Latitude","Longitude","geometry"]]

,Latitude,Longitude,geometry
0,32.5523,-114.9965,POINT (470495.929 -594187.763)
3,34.7783,-118.4323,POINT (143396.998 -358615.563)
4,34.7791,-118.4075,POINT (145663.743 -358488.961)
5,34.7335,-118.2973,POINT (155833.437 -363374.892)
6,34.7086,-118.3041,POINT (155260.597 -366149.588)
...,...,...,...
2769962,31.2105,-115.7500,POINT (406343.478 -745894.014)
2769963,31.2105,-115.7500,POINT (406343.478 -745894.014)
2769964,31.2105,-115.7500,POINT (406343.478 -745894.014)
2769965,31.2105,-115.7500,POINT (406343.478 -745894.014)


In [49]:
#convert coordinates for GOES 16. Running in 1 cell is fine, doesn't kill kernel
coords = [Point(xy) for xy in zip(wfabba_goes_16_df['Longitude'], wfabba_goes_16_df['Latitude'])]
wfabba_goes_16_df = GeoDataFrame(wfabba_goes_16_df, crs = "EPSG:4326", geometry = coords) 
wfabba_goes_16_df = wfabba_goes_16_df.to_crs('EPSG:3310')
wfabba_goes_16_df[["Latitude","Longitude","geometry"]]

,Latitude,Longitude,geometry
1,32.9690,-113.4948,POINT (608345.835 -539578.549)
5,32.9941,-113.5104,POINT (606699.070 -536904.408)
6,32.9921,-113.4776,POINT (609775.877 -536915.028)
7,32.9710,-113.5276,POINT (605268.147 -539567.438)
8,32.9690,-113.4948,POINT (608345.835 -539578.549)
...,...,...,...
384439,34.0118,-117.6624,POINT (215904.527 -442196.275)
384442,34.0118,-117.6624,POINT (215904.527 -442196.275)
384444,33.1127,-115.4588,POINT (424087.357 -534378.062)
384445,34.0118,-117.6624,POINT (215904.527 -442196.275)


In [ ]:
# #matching function for FIgLib events and WFABBA
# def matches_modified(events_df, wfabba_df, minutes_delta, radius_miles):
#     all_matches_df = pd.DataFrame()
#     wfabba_votes = []
#     for row in events_df.iterrows():
#         row = row[1]
        
#         time_lower = wfabba_df["timestamp_converted"] >= (row["datetime"] - timedelta(minutes=minutes_delta))
#         time_upper = wfabba_df["timestamp_converted"] <= (row["datetime"] + timedelta(minutes=minutes_delta))
#         match_results_df = wfabba_df[time_lower & time_upper].copy()
#         match_results_df = match_results_df.reset_index()
#         match_results_df = match_results_df.drop(columns = ["index","level_0"])
        
#         #get rid of low probbability detections
#         not_code_15 = match_results_df["Code"] != 15
#         not_code_35 = match_results_df["Code"] != 35
#         match_results_df = match_results_df[not_code_15 & not_code_35]
#         match_results_df = match_results_df.reset_index()
#         match_results_df = match_results_df.drop(columns="index")
        
#         fig_lib_event_coords = [Point(row["long_adjusted"], row["lat_adjusted"])]*len(match_results_df)
#         fig_lib_event_coords_df = pd.DataFrame(index=range(len(match_results_df)))
#         fig_lib_event_coords_df["Latitude"] = row["lat_adjusted"]
#         fig_lib_event_coords_df["Longitude"] = row["long_adjusted"]
        
#         fig_lib_event_coords_df =  GeoDataFrame(fig_lib_event_coords_df, crs = "EPSG:4326", geometry = fig_lib_event_coords)
#         fig_lib_event_coords_df = fig_lib_event_coords_df.to_crs("EPSG:3310")
        
#         match_results_df["distance"] = match_results_df.distance(fig_lib_event_coords_df)
#         match_results_df["distance"] = match_results_df["distance"]/1609.344
    
#         within_distance = (match_results_df["distance"] <= radius_miles)
        
#         print("FIgLib Event Datetime:",row["datetime"])
#         print("FIgLib Event Latitude:",row["lat_adjusted"]) 
#         print("FIgLib Event Longitude:",row["long_adjusted"])
#         print("")
#         print("WFABBA matches:")
        
#         match_results_df = match_results_df[within_distance]
#         match_results_df = match_results_df.reset_index()
#         match_results_df = match_results_df.drop(columns = ["index"])
#         print(match_results_df)
#         print("******************************************")
        
#         #remove duplicate coords
#         match_results_df = match_results_df.sort_values(by=["timestamp_converted"]).drop_duplicates(subset=["geometry"])
#         match_results_df = match_results_df.reset_index()
#         match_results_df = match_results_df.drop(columns = ["index"])
#         print(match_results_df)
        
#         print(match_results_df)
#         print("+++++++++++++++++++++++++++++++++++")
        
#         match_results_df["Event Lat"] =  row["lat_adjusted"]
#         match_results_df["Event Long"] = row["long_adjusted"]
#         match_results_df["Event Datetime"] = row["datetime"]
#         match_results_df["Lat Diff"] = (match_results_df["Latitude"] - match_results_df["Event Lat"]).abs()
#         match_results_df["Long Diff"] = (match_results_df["Longitude"] - match_results_df["Event Long"]).abs()
#         match_results_df["Time Diff"] = (match_results_df["timestamp_converted"] - match_results_df["Event Datetime"]).dt.total_seconds()/60
#         match_results_df["Event Name"] = row["event_name"][:-1]
        
#         print(match_results_df[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff","distance"]])
#         all_matches_df = pd.concat([all_matches_df, match_results_df])
#         # print(all_matches_df)
#         print("============================")
#         all_matches_df = all_matches_df.reset_index()
#         all_matches_df = all_matches_df.drop(columns=["index"])
        
#         if len(match_results_df) > 0:
#             vote = 1
#         else:
#             vote = 0
            
#         wfabba_votes.append(vote)
    
#     return all_matches_df, wfabba_votes
#     # break


In [ ]:
# #matching WFABBA GOES 16 to FIgLib Events

# %%time
# goes16_matches = matches_modified(overlap_events_df, wfabba_goes_16_df, 30, 35)
# goes16_matches[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]]
# # goes16_matches.to_csv("goes-16_30_35.csv")
# goes16_matches

In [ ]:
# #matching WFABBA GOES 17 to FIgLib Events

# %%time
# goes17_matches = matches_modified(overlap_events_df, wfabba_goes_17_df, 30, 35)
# goes17_matches[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]]
# # goes17_matches.to_csv("goes-17_30_35.csv")
# goes17_matches

## 4) Matching WFABBA to SmokeyNet 

In [50]:
#Create dataframe for every minute of 2020

# start_datetime = '2020-01-01 00:00:00'
# end_dateime = '2020-12-31 23:59:59'

times = []

# start = datetime(2021, 1 , 1, 0, 0, 0, 0, pytz.UTC)
# end = datetime(2020, 12, 31, 23, 59, 0, 0, pytz.UTC)
# start = datetime(2020, 2 , 2, 0, 0, 0, 0, pytz.UTC)
start = datetime(2019, 6 , 1, 0, 0, 0, 0, pytz.UTC)
# end = datetime(2020, 12, 31, 23, 59, 0, 0, pytz.UTC)
end = datetime(2021, 7, 11, 23, 59, 0, 0, pytz.UTC)
# end = datetime(2021, 7, 11, 23, 59, 0, 0, pytz.UTC)
# end = datetime(2019, 12, 31, 23, 59, 0, 0, pytz.UTC)

while start <= end:
    times.append(start)
    start = start + timedelta(minutes = 1)

minutes_df = pd.DataFrame(times, columns = ["timestamp"])
minutes_df

,timestamp
0,2019-06-01 00:00:00+00:00
1,2019-06-01 00:01:00+00:00
2,2019-06-01 00:02:00+00:00
3,2019-06-01 00:03:00+00:00
4,2019-06-01 00:04:00+00:00
...,...
1111675,2021-07-11 23:55:00+00:00
1111676,2021-07-11 23:56:00+00:00
1111677,2021-07-11 23:57:00+00:00
1111678,2021-07-11 23:58:00+00:00


In [ ]:
# # Create training SmokeyNet df 
# df_train = pd.read_json("train_data.json", orient="index").reset_index().rename(columns={"index":"filepath"})
# df_train["type"] = "train"
# df_train 

In [51]:
# Create testing SmokeyNet df
df_test = pd.read_json("test_data.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_test["type"] = "test"
df_test 

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test
...,...,...,...,...,...,...,...
4880,20161113_FIRE_bm-n-mobo-c/1479069033_+02100,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4881,20161113_FIRE_bm-n-mobo-c/1479069093_+02160,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4882,20161113_FIRE_bm-n-mobo-c/1479069153_+02220,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4883,20161113_FIRE_bm-n-mobo-c/1479069213_+02280,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test


In [52]:
df_test["camera_name"].unique()

array(['20191001_FIRE_lp-s-mobo-c', '20201105_Roundfire_pi-s-mobo-c',
       '20200306_FIRE_pi-n-mobo-c', '20190922_FIRE_ml-w-mobo-c',
       '20191006_FIRE_lo-s-mobo-c', '20200806_BorderFire_om-e-mobo-c',
       '20200905_ValleyFire_lp-n-mobo-c',
       '20200829_inside-Mexico_mlo-s-mobo-c',
       '20190610_Pauma_bh-w-mobo-c', '20200905_ValleyFire_sm-e-mobo-c',
       '20200828_BorderFire_sm-s-mobo-c', '20190913_FIRE_lp-n-mobo-c',
       '20190801_Caliente_om-w-mobo', '20200206_FIRE_ml-s-mobo-c',
       '20160718_FIRE_lp-n-iqeye', '20191007_FIRE_om-s-mobo-c',
       '20200829_inside-Mexico_om-e-mobo-c', '20200705_FIRE_bm-w-mobo-c',
       '20190827_FIRE_so-w-mobo-c', '20210209_FIRE_hp-e-mobo-c',
       '20210107_Miguelfire_om-w-mobo-c', '20180728_FIRE_rm-w-mobo-c',
       '20200812_LakeFire_dwpgm-n-mobo-c', '20191001_FIRE_bh-w-mobo-c',
       '20200709_Tripp_hp-n-mobo-c', '20200615_Rainbow_rm-e-mobo-c',
       '20200829_inside-Mexico_cp-s-mobo-c',
       '20210113_Borderfire_mlo-s-mo

In [53]:
#Create validating SmokeyNet df
df_valid = pd.read_json("valid_data.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_valid["type"] = "valid"
df_valid

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
1,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
2,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
3,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
...,...,...,...,...,...,...,...
4906,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4907,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4908,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4909,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid


In [54]:
#Join the SmokeyNet DFs together. For now just joining validation and test DFs

# df_labels = pd.concat([df_train, df_test, df_valid]).reset_index().drop(columns = ["index"])
df_labels = pd.concat([df_test, df_valid]).reset_index().drop(columns = ["index"])
df_labels = pd.concat([df_test, df_valid]).reset_index().drop(columns = ["index"])
df_labels

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test
...,...,...,...,...,...,...,...
9791,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
9792,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
9793,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
9794,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid


In [55]:
# set the date and year columns
df_labels["date"] = df_labels["camera_name"].str.split("_", n=1, expand=True)[0]
df_labels["year"] = df_labels["date"].str[:4]
df_labels["date"] = pd.to_datetime(df_labels["date"])
df_labels

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019
...,...,...,...,...,...,...,...,...,...
9791,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
9792,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
9793,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
9794,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020


In [56]:
#print the DeLuzFire SmokeyNet detections
df_labels[df_labels["camera_name"] == "20200930_DeLuzFire_rm-w-mobo-c"]

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
3264,20200930_DeLuzFire_rm-w-mobo-c/1601492466_-01140,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2020-09-30,2020
3265,20200930_DeLuzFire_rm-w-mobo-c/1601492526_-01080,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2020-09-30,2020
3266,20200930_DeLuzFire_rm-w-mobo-c/1601492586_-01020,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2020-09-30,2020
3267,20200930_DeLuzFire_rm-w-mobo-c/1601492646_-00960,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2020-09-30,2020
3268,20200930_DeLuzFire_rm-w-mobo-c/1601492706_-00900,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2020-09-30,2020
3269,20200930_DeLuzFire_rm-w-mobo-c/1601492766_-00840,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2020-09-30,2020
3270,20200930_DeLuzFire_rm-w-mobo-c/1601492826_-00780,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2020-09-30,2020
3271,20200930_DeLuzFire_rm-w-mobo-c/1601492886_-00720,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2020-09-30,2020
3272,20200930_DeLuzFire_rm-w-mobo-c/1601492946_-00660,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2020-09-30,2020
3273,20200930_DeLuzFire_rm-w-mobo-c/1601493006_-00600,20200930_DeLuzFire_rm-w-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2020-09-30,2020


#Pseudo code
We have every minute of 2020, picked out 1 camera
1) Create 2020 minutes df
2) Pic 1 camera and snap datetime to upper minute
3) Join the minute df with camera events
4) Filter both WFABBAs for events just within location proximity and direction
5) Same minute snap for WFABBAs and join to minute df (Consider multiple detections in the same direction and within location prox as 1 detection)
    Caveat: Initially keep repeats (later on maybe get first instance and autofill for the next x minutes)
6) Join with minute df
7) Sum each row's prediciton. Must be at least 2



In [57]:
# df_labels_filtered = df_labels[df_labels["date"] >= "2021-01-01"].reset_index().drop(columns=["index"])
# df_labels_filtered = df_labels[df_labels["date"] >= "2020-01-01"].reset_index().drop(columns=["index"])
df_labels_filtered = df_labels[df_labels["date"] >= "2019-06-01"].reset_index().drop(columns=["index"])
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019
...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020


In [58]:
df_labels_filtered["time"] = df_labels_filtered["filepath"].str.split("/").str[1]
df_labels_filtered["time"] = df_labels_filtered["time"].str.split("_").str[0]
df_labels_filtered["datetime"] = pd.to_datetime(df_labels_filtered["time"], unit="s", origin="unix", utc=True)
df_labels_filtered["event_name"] = df_labels_filtered["filepath"].str.split("/").str[0]

# df_labels_filtered["date"] = df_labels_filtered["filepath"].str[:8]
df_labels_filtered["camera_name"] = df_labels_filtered["camera_name"].str.split("_").str[-1]

#Edge case: 2 camera names not following naming convention. Uses "-" instead of "_"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "FIRE-pi-s-mobo-c", ["camera_name"]] = "pi-s-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "FIRE-smer-tcs8-mobo-c", ["camera_name"]] = "smer-tcs8-mobo-c"

df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c
...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c


In [59]:
df_labels_filtered = df_labels_filtered.merge(camera_metadata_filtered_df, left_on="camera_name", right_on="camera_name", how="left")
df_labels_filtered.loc[df_labels_filtered["lat_adjusted"].isna(), ["lat_adjusted"]] = df_labels_filtered["lat"]
df_labels_filtered.loc[df_labels_filtered["long_adjusted"].isna(), ["long_adjusted"]] = df_labels_filtered["long"]
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,lat,long,lat_adjusted,long_adjusted
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810


In [76]:
len(df_labels_filtered)

8825

In [60]:
len(df_labels_filtered[df_labels_filtered["type"]=="test"])

4378

In [61]:
len(df_labels_filtered[df_labels_filtered["type"]=="test"]["event_name"].unique())

56

In [62]:
len(df_labels_filtered[df_labels_filtered["type"]=="valid"]["event_name"].unique())

58

In [64]:
len(df_labels_filtered["event_name"].unique())

114

In [65]:
coords = [Point(xy) for xy in zip(df_labels_filtered['long_adjusted'], df_labels_filtered['lat_adjusted'])]
df_labels_filtered = GeoDataFrame(df_labels_filtered, crs = "EPSG:4326", geometry = coords) 
df_labels_filtered = df_labels_filtered.to_crs('EPSG:3310')
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,lat,long,lat_adjusted,long_adjusted,geometry
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039)
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039)
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039)
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039)
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575)
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575)
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575)
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575)


In [66]:
df_labels_filtered.loc[df_labels_filtered["camera_name"].str.split("-").str[1] == 'n','direction'] = 'north'
df_labels_filtered.loc[df_labels_filtered["camera_name"].str.split("-").str[1] == 's','direction'] = 'south'
df_labels_filtered.loc[df_labels_filtered["camera_name"].str.split("-").str[1] == 'e','direction'] = 'east'
df_labels_filtered.loc[df_labels_filtered["camera_name"].str.split("-").str[1] == 'w','direction'] = 'west'
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,lat,long,lat_adjusted,long_adjusted,geometry,direction
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north


In [67]:
def round_secs(x):
    x = x + timedelta(minutes = 1)
    x = x.replace(second=0)
    return x

In [68]:
def is_in_camera_direction(camera_geometry_pt, direction, wfabba_geometry_pt):
    if direction == "north":
        return wfabba_geometry_pt.y >= camera_geometry_pt.y
    elif direction == "south":
        return wfabba_geometry_pt.y <= camera_geometry_pt.y
    elif direction == "east":
        return wfabba_geometry_pt.x >= camera_geometry_pt.x
    elif direction == "west":
        return wfabba_geometry_pt.x <= camera_geometry_pt.x
    else:
        # unknown or something else
        pass
    

In [69]:
def matches_distance_prox(camera_geometry, direction, radius_miles, wfabba_df):
# def matches_distance_prox(camera_geometry, radius_miles, wfabba_df):

    wfabba_df["distance_m"] = wfabba_df["geometry"].distance(camera_geometry)
    wfabba_df["distance_mi"] = wfabba_df["distance_m"]/1609.344        
    match_results_df = wfabba_df[(wfabba_df["distance_mi"] <= radius_miles)].copy()
    
    
    #filter for detections within same direction
    match_results_df["is_in_direction"] = match_results_df.apply(
        lambda row: is_in_camera_direction(camera_geometry, direction, row["geometry"]), axis=1
    )
    match_results_df = match_results_df[match_results_df["is_in_direction"] == True]

    return match_results_df

In [70]:
df_labels_filtered["datetime_rounded"] = df_labels_filtered["datetime"].apply(lambda x: round_secs(x))
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,lat,long,lat_adjusted,long_adjusted,geometry,direction,datetime_rounded
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south,2019-10-01 17:22:00+00:00
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south,2019-10-01 17:23:00+00:00
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south,2019-10-01 17:24:00+00:00
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south,2019-10-01 17:25:00+00:00
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,32.7015,-116.7646,32.7015,-116.7646,POINT (303754.080 -584902.039),south,2019-10-01 17:26:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north,2020-08-13 23:08:00+00:00
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north,2020-08-13 23:09:00+00:00
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north,2020-08-13 23:10:00+00:00
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,33.5191,-117.4810,33.5191,-117.4810,POINT (234106.615 -496384.575),north,2020-08-13 23:11:00+00:00


In [71]:
df_labels_filtered["date"].sort_values().unique()

array(['2019-06-10T00:00:00.000000000', '2019-07-12T00:00:00.000000000',
       '2019-07-14T00:00:00.000000000', '2019-07-16T00:00:00.000000000',
       '2019-08-01T00:00:00.000000000', '2019-08-03T00:00:00.000000000',
       '2019-08-09T00:00:00.000000000', '2019-08-13T00:00:00.000000000',
       '2019-08-14T00:00:00.000000000', '2019-08-25T00:00:00.000000000',
       '2019-08-26T00:00:00.000000000', '2019-08-27T00:00:00.000000000',
       '2019-08-29T00:00:00.000000000', '2019-09-13T00:00:00.000000000',
       '2019-09-15T00:00:00.000000000', '2019-09-22T00:00:00.000000000',
       '2019-09-24T00:00:00.000000000', '2019-09-25T00:00:00.000000000',
       '2019-09-30T00:00:00.000000000', '2019-10-01T00:00:00.000000000',
       '2019-10-03T00:00:00.000000000', '2019-10-05T00:00:00.000000000',
       '2019-10-06T00:00:00.000000000', '2019-10-07T00:00:00.000000000',
       '2020-02-02T00:00:00.000000000', '2020-02-06T00:00:00.000000000',
       '2020-02-26T00:00:00.000000000', '2020-03-04

In [72]:
unique_cameras = df_labels_filtered["camera_name"].unique()
unique_cameras

array(['lp-s-mobo-c', 'pi-s-mobo-c', 'pi-n-mobo-c', 'ml-w-mobo-c',
       'lo-s-mobo-c', 'om-e-mobo-c', 'lp-n-mobo-c', 'mlo-s-mobo-c',
       'bh-w-mobo-c', 'sm-e-mobo-c', 'sm-s-mobo-c', 'om-w-mobo',
       'ml-s-mobo-c', 'om-s-mobo-c', 'bm-w-mobo-c', 'so-w-mobo-c',
       'hp-e-mobo-c', 'om-w-mobo-c', 'dwpgm-n-mobo-c', 'hp-n-mobo-c',
       'rm-e-mobo-c', 'cp-s-mobo-c', 'tp-s-mobo-c', 'smer-tcs8-mobo-c',
       'wc-e-mobo-c', 'wc-n-mobo-c', 'rm-w-mobo-c', 'smer-tcs9-mobo-c',
       'wc-s-mobo-c', 'sclm-e-mobo-c', 'mg-n-mobo-c', 'lp-w-mobo-c',
       'hp-s-mobo-c', 'mlo-n-mobo-c', 'pi-s-mobo', 'pi-e-mobo-c',
       'om-n-mobo-c', 'sp-n-mobo-c', 'om-s-mobo', 'sm-w-mobo-c',
       'rm-n-mobo-c', 'hp-w-mobo-c', 'tp-w-mobo-c', '69bravo-e-mobo-c',
       'pi-w-mobo-c', 'sjh-n-mobo-c', 'bl-n-mobo-c', 'lp-s-mobo',
       'syp-w-mobo-c', 'sm-n-mobo-c', 'vo-n-mobo-c', 'lp-e-mobo-c',
       'bm-e-mobo-c', 'bl-s-mobo-c', 'marconi-n-mobo-c'], dtype=object)

In [73]:
unique_cameras = ['lp-s-mobo-c', 'pi-s-mobo-c', 'pi-n-mobo-c', 'ml-w-mobo-c',
       'lo-s-mobo-c', 'om-e-mobo-c', 'lp-n-mobo-c', 'mlo-s-mobo-c',
       'bh-w-mobo-c', 'sm-e-mobo-c', 'sm-s-mobo-c', 'om-w-mobo',
       'ml-s-mobo-c', 'om-s-mobo-c', 'bm-w-mobo-c', 'so-w-mobo-c',
       'hp-e-mobo-c', 'om-w-mobo-c', 'dwpgm-n-mobo-c', 'hp-n-mobo-c',
       'rm-e-mobo-c', 'cp-s-mobo-c', 'tp-s-mobo-c', 
       'wc-e-mobo-c', 'wc-n-mobo-c', 'rm-w-mobo-c',
       'wc-s-mobo-c', 'sclm-e-mobo-c', 'mg-n-mobo-c', 'lp-w-mobo-c',
       'hp-s-mobo-c', 'mlo-n-mobo-c', 'pi-s-mobo', 'pi-e-mobo-c',
       'om-n-mobo-c', 'sp-n-mobo-c', 'om-s-mobo', 'sm-w-mobo-c',
       'rm-n-mobo-c', 'hp-w-mobo-c', 'tp-w-mobo-c', '69bravo-e-mobo-c',
       'pi-w-mobo-c', 'sjh-n-mobo-c', 'bl-n-mobo-c', 'lp-s-mobo',
       'syp-w-mobo-c', 'sm-n-mobo-c', 'vo-n-mobo-c', 'lp-e-mobo-c',
       'bm-e-mobo-c', 'bl-s-mobo-c', 'marconi-n-mobo-c']

In [74]:
%%time
#looping for each camera station
for camera in unique_cameras:
    print("Camera:",camera)
    camera_df = df_labels_filtered[df_labels_filtered["camera_name"].str.contains(camera)].copy()

    #Find GOES-16 matches
    goes_16_dist_match_df = matches_distance_prox(camera_df.iloc[0]["geometry"], camera_df.iloc[0]["direction"], 35, wfabba_goes_16_df)
    goes_16_dist_match_df["timestamp_converted_rounded"] = goes_16_dist_match_df["timestamp_converted"].apply(lambda x: round_secs(x))
    goes_16_dist_match_df = goes_16_dist_match_df.drop_duplicates(subset = ["timestamp_converted_rounded"])

    #Find GOES-17 matches
    goes_17_dist_match_df = matches_distance_prox(camera_df.iloc[0]["geometry"],  camera_df.iloc[0]["direction"], 35, wfabba_goes_17_df)
    goes_17_dist_match_df["timestamp_converted_rounded"] = goes_17_dist_match_df["timestamp_converted"].apply(lambda x: round_secs(x))
    goes_17_dist_match_df = goes_17_dist_match_df.drop_duplicates(subset = ["timestamp_converted_rounded"])

    #SmokeyNet_join
    test_df = minutes_df.merge(camera_df, left_on = "timestamp", right_on = "datetime_rounded",how="left")
    test_df = test_df.rename(columns = {"geometry":"HPWREN_Station_geometry", "lat_adjusted":"HPWREN_lat", "long_adjusted":"HPWREN_long", "datetime_rounded":"SmokeyNet_datetime_rounded"})
    print("joined SmokeyNet")
    
    #GOES-16 Join
    test_df = test_df.merge(goes_16_dist_match_df[["timestamp_converted_rounded", "geometry"]], left_on = "timestamp", right_on = "timestamp_converted_rounded",how="left")
    test_df = test_df.rename(columns = {"geometry":"WFABBA_GOES16_geometry", "timestamp_converted_rounded":"WFABBA_GOES16_timestamp_converted_rounded"})
#     test_df = test_df[["timestamp","camera_name", "image_gt", "image_pred", "type", "WFABBA_GOES16_geometry", "WFABBA_GOES16_distance", "HPWREN_Station_geometry", "HPWREN_lat", "HPWREN_long"]]
    test_df = test_df[["timestamp","camera_name", "image_gt", "image_pred", "type", "WFABBA_GOES16_geometry"]]
                                                  
    test_df.loc[test_df["WFABBA_GOES16_geometry"] != None,'goes16_pred'] = 1
    test_df.loc[test_df["WFABBA_GOES16_geometry"] == None,'goes16_pred'] = 0
    print("joined GOES16")


    #GOES-17 Join
    test_df = test_df.merge(goes_17_dist_match_df[["timestamp_converted_rounded", "geometry"]], left_on = "timestamp", right_on = "timestamp_converted_rounded",how="left")
    test_df = test_df.rename(columns = {"geometry":"WFABBA_GOES17_geometry", "timestamp_converted_rounded":"WFABBA_GOES17_timestamp_converted_rounded"})
#     test_df = test_df[["timestamp","camera_name", "image_gt", "image_pred", "type", "WFABBA_GOES16_geometry", "WFABBA_GOES16_distance", "goes16_pred", "HPWREN_Station_geometry", "HPWREN_lat", "HPWREN_long", "WFABBA_GOES17_geometry", "WFABBA_GOES17_distance"]]
    test_df = test_df[["timestamp","camera_name", "image_gt", "image_pred", "type", "WFABBA_GOES16_geometry", "goes16_pred", "WFABBA_GOES17_geometry"]]
    test_df.loc[test_df["WFABBA_GOES17_geometry"] != None,'goes17_pred'] = 1
    test_df.loc[test_df["WFABBA_GOES17_geometry"] == None,'goes17_pred'] = 0
    print("joined GOES17")


    #Get all votes and determine if smoke was detected by majority rule
    test_df["final_vote"] = test_df["image_pred"] + test_df["goes16_pred"] + test_df["goes17_pred"]
    test_df.loc[test_df["final_vote"] >= 2,'final_pred'] = 1
    test_df.loc[test_df["final_vote"] < 2,'final_pred'] = 0

#     print(test_df[~test_df["image_gt"].isna()][["timestamp","image_gt", "image_pred", "goes16_pred", "goes17_pred", "final_pred"]])

    image_labels = test_df[~test_df["image_gt"].isna()]["image_gt"]
    smokeynet_preds = test_df[~test_df["image_gt"].isna()]["image_pred"]
    ensemble_preds = test_df[~test_df["image_gt"].isna()]["final_pred"]

    baseline_score = accuracy_score(image_labels, smokeynet_preds)
    ensemble_score = accuracy_score(image_labels, ensemble_preds)
    
    print("Baseline score:", baseline_score)
    print("Ensemble score:", ensemble_score)
    test_df[~test_df["image_gt"].isna()][["timestamp","image_gt", "image_pred", "goes16_pred", "goes17_pred", "final_pred","type"]].to_csv(camera + "_all_hard_voting.csv")
#     test_df.to_csv(camera + "_2019_hard_voting.csv")
#     test_df.to_csv(camera + "_hard_voting.csv")
    print("=====================================================")

Camera: lp-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.7095238095238096
Ensemble score: 0.5142857142857142
Camera: pi-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.8805031446540881
Ensemble score: 0.5513626834381551
Camera: pi-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.7
Ensemble score: 0.4875
Camera: ml-w-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9288702928870293
Ensemble score: 0.5355648535564853
Camera: lo-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.8235294117647058
Ensemble score: 0.6512605042016807
Camera: om-e-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.6005025125628141
Ensemble score: 0.5879396984924623
Camera: lp-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9002217294900222
Ensemble score: 0.5764966740576497
Camera: mlo-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 

joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.8875
Ensemble score: 0.5875
Camera: sm-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9161290322580645
Ensemble score: 0.5548387096774193
Camera: vo-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.7105263157894737
Ensemble score: 0.5263157894736842
Camera: lp-e-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.925
Ensemble score: 0.4875
Camera: bm-e-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9358974358974359
Ensemble score: 0.5
Camera: bl-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9230769230769231
Ensemble score: 0.6282051282051282
Camera: marconi-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.6111111111111112
Ensemble score: 0.5
CPU times: user 20min 59s, sys: 33.5 s, total: 21min 32s
Wall time: 21min 33s


## NOTE: Old logic for joining WFABBA detections with SmokeyNet. 
Only run match when there is a positive SmokeyNet prediction

In [ ]:
df_labels_filtered_positive_pred = df_labels_filtered[df_labels_filtered["image_pred"] == 1]
df_labels_filtered_positive_pred = df_labels_filtered_positive_pred.reset_index()
df_labels_filtered_positive_pred = df_labels_filtered_positive_pred.drop(columns=["index"])
df_labels_filtered_positive_pred

In [ ]:
%%time
# goes16_matches = matches(overlap_events_df, wfabba_goes_16_df, 30, 35)
goes16_matches, goes16_votes = matches_modified(df_labels_filtered_positive_pred, wfabba_goes_16_df, 720, 35)
goes16_matches[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]]
goes16_matches

In [ ]:
goes16_matches

In [ ]:
goes16_votes_df = pd.DataFrame(goes16_votes)
goes16_votes_df = goes16_votes_df.rename(columns={0:"Vote"})
goes16_votes_df

In [ ]:
%%time
goes17_matches, goes17_votes = matches_modified(df_labels_filtered_positive_pred, wfabba_goes_17_df, 720, 35)
goes17_matches[["Latitude","Longitude", "timestamp_converted", "Event Lat", "Event Long", "Event Datetime", "Lat Diff", "Long Diff", "Time Diff"]]
goes17_matches

In [ ]:
goes17_matches

In [ ]:
goes17_votes_df = pd.DataFrame(goes17_votes)
goes17_votes_df = goes17_votes_df.rename(columns={0:"Vote"})
goes17_votes_df

In [ ]:
final_votes_list = []
for i in range(len(df_labels_filtered_positive_pred)):
    vote_sum = 0
    vote_sum += df_labels_filtered_positive_pred.iloc[i]["image_pred"]
    vote_sum += goes16_votes_df.iloc[i]["Vote"]
    vote_sum += goes17_votes_df.iloc[i]["Vote"]
    if vote_sum >= 2:
        final_decision = 1
    else:
        final_decision = 0
    final_votes_list.append(final_decision)

final_votes_df = pd.DataFrame(final_votes_list, columns=["Vote"])

In [ ]:
final_votes_df.value_counts()

In [ ]:
df_labels_filtered_positive_pred["image_gt"].value_counts()

In [ ]:
df_labels_filtered_positive_pred["image_pred"].value_counts()

In [ ]:
compare_votes_df = df_labels_filtered_positive_pred["image_pred"] == goes16_votes_df["Vote"]
compare_votes_df

In [ ]:
compare_votes_df.value_counts()

In [ ]:
df_labels_filtered_positive_pred["image_gt"].value_counts()

In [ ]:
goes16_matches["Event Name"].unique()

In [ ]:
df_labels_filtered_positive_pred["event_name"].unique()

In [ ]:
df_labels_filtered_positive_pred["datetime"].describe()

In [ ]:
# TODO - FIgLib Cleanup: 
# 1) Update the event names to use "_" instead of "-" as the separator
# ex: "FIRE-pi-s-mobo-c", "FIRE-smer-tcs8-mobo-c"
# 
# 2) 9 FIgLib events that lacked a pic with +00000.jpg. 
# ex: ['20170708_Whittier_syp-n-mobo-m/',
# '20180809_FIRE_bh-s-mobo-c/',
# '20190610_Pauma_bh-w-mobo-c/',
# '20190610_Pauma_bh-w-mobo-m/',
# '20200828_BorderFire_om-w-mobo-c/',
# '20200831_FIRE_wc-n-mobo-c/',
# '20200905_ValleyFire_cp-s-mobo-c/',
# '20201202_WillowFire-nightime-near-CDF-HQ_lp-w-mobo-c/',
# '20210110_Borderfire_lp-s-mobo-c/']